# Implementing Support Vector Machines with graph kernel

In [70]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from scipy import optimize
import networkx as nx
from hashlib import blake2b
from collections import Counter, defaultdict
import itertools

In [133]:
with open("./data-challenge-kernel-methods-2022-2023/training_data.pkl", "rb") as f:
    train_data = pickle.load(f)

with open("./data-challenge-kernel-methods-2022-2023/training_labels.pkl", "rb") as f:
    train_labels = pickle.load(f)

In [136]:
train_labels_svm = train_labels.copy()
train_labels_svm[train_labels_svm == 0] = -1
np.unique(train_labels_svm)

array([-1,  1])

In [154]:
class KernelSVC:
    
    def __init__(self, C, kernel_mat, epsilon = 1e-3):
        self.type = 'non-linear'
        self.C = C                               
        self.kernel = kernel_mat        
        self.alpha = None
        self.support = None
        self.epsilon = epsilon
        self.norm_f = None
       
    
    def fit(self, X, y):
       #### You might define here any variable needed for the rest of the code
        N = len(y)
        K = self.kernel(X, X)
        
        # Store training data
        self.X_train = X
        self.obs_train = y

        # define matrix form of inequality constraints on dual problem
        ineq_A = np.vstack([np.eye(N), -np.eye(N)])
        ineq_b = np.array([self.C]*N + [0]*N)

        # Lagrange dual problem
        def loss(alpha):
            # The dual problem is a max pb => max f = min -f
            return .5*(alpha*y).T@K@(alpha*y) - alpha.sum()

        # Partial derivate of Ld on alpha
        def grad_loss(alpha):
            return y*((alpha*y)@K) - 1


        # Constraints on alpha of the shape :
        # -  d - C*alpha  = 0
        # -  b - A*alpha >= 0

        fun_eq = lambda alpha: alpha@y
        jac_eq = lambda alpha: y
        fun_ineq = lambda alpha: ineq_b - ineq_A@alpha
        jac_ineq = lambda alpha: -ineq_A
        
        constraints = ({'type': 'eq',  'fun': fun_eq, 'jac': jac_eq},
                       {'type': 'ineq', 
                        'fun': fun_ineq , 
                        'jac': jac_ineq})

        optRes = optimize.minimize(fun=lambda alpha: loss(alpha),
                                   x0=np.ones(N), 
                                   method='SLSQP', 
                                   jac=lambda alpha: grad_loss(alpha), 
                                   constraints=constraints)
        self.alpha = optRes.x

        ## Assign the required attributes
        margin_idx = (self.alpha >= self.epsilon)
        sv_idx = (self.alpha >= self.epsilon) * (self.C - self.alpha >= self.epsilon)
        self.support = X[sv_idx]
        self.margin_points = X[margin_idx]
        self.margin_obs = y[margin_idx]
        self.margin_alpha = self.alpha[margin_idx]
        self.support_obs = y[sv_idx]
        self.support_alpha = self.alpha[sv_idx]
        
        # Take mean of offsets for more robust approximation
        self.b = np.mean(y[sv_idx] - self.separating_function(self.support))
        self.norm_f = np.sqrt(self.margin_alpha@self.kernel(self.margin_points, self.margin_points)@self.margin_alpha)


    ### Implementation of the separting function $f$ 
    def separating_function(self,x):
        # Input : matrix x of shape N data points times d dimension
        # Output: vector of size N
        
        return (self.support_alpha*self.support_obs).T@self.kernel(self.support, x)
    
    
    def predict(self, X):
        """ Predict y values in {-1, 1} """
        d = self.separating_function(X)
        return 2*(d+self.b> 0) - 1

In [189]:
class RandomWalkKernel:
    def __init__(self, l_walk, n_walks):
        self.l = l_walk
        self.n_walks = n_walks
    
    def perform_random_walk(self, edge_list, g, rdm_node):
        curr_node = rdm_node
        s = ''
        for _ in range(self.l + 1):
            s += str(g.nodes[curr_node]['labels'][0])
            # Undirected graph !
            sample_neighbor = edge_list.loc[(edge_list['source'] == curr_node) + (edge_list["target"] == curr_node), :]
            if len(sample_neighbor) == 0:
                continue
            else:
                sample_neighbor = sample_neighbor.sample()
            # Get new node
            if sample_neighbor["source"].values == curr_node:
                next_node = sample_neighbor["target"].values[0]
            else:
                next_node = sample_neighbor["source"].values[0]

            s+= str(sample_neighbor["labels"].values[0][0])
            curr_node = next_node  
        return s
        
    def compute_kernel(self, g0, g1):
        
        # Store edge list
        edges_g0 = nx.to_pandas_edgelist(g0)
        edges_g1 = nx.to_pandas_edgelist(g1)

        # Run random walks
        seq_g0 = []
        seq_g1 = []
        for _ in range(self.n_walks):
            rdm_node_g0 = np.random.choice(g0.nodes)
            seq_g0.append(self.perform_random_walk(edges_g0, g0, rdm_node_g0))

            rdm_node_g1 = np.random.choice(g1.nodes)
            seq_g1.append(self.perform_random_walk(edges_g1, g1, rdm_node_g1))

        # Compute kernel
        keys, vals = np.unique(seq_g1, return_counts=True)
        dico_g1 = dict(zip(list(keys), list(vals)))

        keys, vals = np.unique(seq_g0, return_counts=True)
        dico_g0 = dict(zip(list(keys), list(vals)))
        common_occ = 0
        for good_seq in np.intersect1d(seq_g0, seq_g1):
            common_occ += min(int(dico_g0[good_seq]), int(dico_g1[good_seq]))
        
        common_occ /= self.n_walks
        
        return common_occ
    
    def compute_kernel_matrix(self, X, Y):
        k = np.zeros((len(X), len(Y)))
        for i in range(len(X)):
            for j in range(i, len(Y)):
                k[i, j] = self.compute_kernel(X[i], Y[j]) 
                k[j, i] = k[i,j]
        return k

In [218]:
class WLKernel:
    def __init__(self, edge_attr="labels", node_attr="labels", iterations=3):
        self.edge_attr = edge_attr
        self.node_attr = node_attr
        self.n_iter = iterations
    
    def _hash_label(self, label, digest_size):
        return blake2b(label.encode("ascii"), digest_size=digest_size).hexdigest()

    def _neighborhood_aggregate(self, G, node, node_labels):
        """
        Compute new labels for given node by aggregating
        the labels of each node's neighbors.
        """
        label_list = []
        for nbr in G.neighbors(node):
            prefix = "" if self.edge_attr is None else str(G[node][nbr][self.edge_attr])
            label_list.append(prefix + node_labels[nbr])
        return node_labels[node] + "".join(sorted(label_list))

    def weisfeiler_lehman_graph_hash(self, G, digest_size=16):
        def weisfeiler_lehman_step(G, labels):
            """
            Apply neighborhood aggregation to each node
            in the graph.
            Computes a dictionary with labels for each node.
            """
            new_labels = {}
            for node in G.nodes():
                label = self._neighborhood_aggregate(G, node, labels)
                new_labels[node] = self._hash_label(label, digest_size)
            return new_labels

        # set initial node labels
        node_labels = {u: str(dd[self.node_attr]) for u, dd in G.nodes(data=True)}

        subgraph_hash_counts = {}
        for it in range(self.n_iter):
            node_labels = weisfeiler_lehman_step(G, node_labels)
            counter = Counter(node_labels.values())
            # normalize counter
            total = np.sum(list(counter.values()))
            for k in counter:
                counter[k] /= total

            # sort the counter, extend total counts
            subgraph_hash_counts[it] = sorted(counter.items(), key=lambda x: x[0])

        # return _hash_label(str(tuple(subgraph_hash_counts)), digest_size)
        return subgraph_hash_counts
    
    
    def compute_phi(self, Z):
        phi_list = []
        for g in Z:
            phi_list.append(self.weisfeiler_lehman_graph_hash(g))
        return phi_list
    
    def compute_kernel(self, wl1, wl2):
        k = 0
        for i in range(self.n_iter):
            dict1 = dict(wl1[i])
            dict2 = dict(wl2[i])
            # take scalar product only on common keys
            common_keys = set(dict1.keys()).intersection(set(dict2.keys()))
            k += np.sum([dict1[c]*dict2[c] for c in common_keys])
        return k

    def compute_kernel_matrix(self, X, Y):
        # Precompute phi to deal only with dot products
        phi_X = self.compute_phi(X)
        if np.array_equal(X, Y):
            print("Not computing phi again as X=Y")
            phi_Y = phi_X.copy()
        else:
            phi_Y = self.compute_phi(Y)
        ker = np.zeros((len(X), len(Y)))
        count_iter = 0
        if len(X) == len(Y):
            for i in range(len(X)):
                for j in range(i, len(Y)):
                    ker[i, j] = self.compute_kernel(phi_X[i], phi_Y[j])
                    ker[j, i] = ker[i,j]
                count_iter += 1
                if count_iter % 100 == 0:
                    print(f"Iteration {count_iter}")
        else:
            for (i,j) in itertools.product(range(len(X)), range(len(Y))):
                ker[i,j] = self.compute_kernel(phi_X[i], phi_Y[j])
        print("Kernel computed")
        return ker

In [ ]:
C = 100
kernel = WLKernel(iterations=5).compute_kernel_matrix
model = KernelSVC(C=C, kernel_mat=kernel)
model.fit(np.array(train_data[:5000], dtype=object), np.array(train_labels_svm[:5000], dtype=object))

Not computing phi again as X=Y
Iteration 100
Iteration 200
Iteration 300
Iteration 400
Iteration 500
Iteration 600
Iteration 700
Iteration 800
Iteration 900
Iteration 1000
Iteration 1100
Iteration 1200
Iteration 1300
Iteration 1400
Iteration 1500
Iteration 1600
Iteration 1700
Iteration 1800
Iteration 1900
Iteration 2000
Iteration 2100
Iteration 2200
Iteration 2300
Iteration 2400
Iteration 2500
Iteration 2600
Iteration 2700
Iteration 2800
Iteration 2900
Iteration 3000
Iteration 3100
Iteration 3200
Iteration 3300
Iteration 3400
Iteration 3500
Iteration 3600
Iteration 3700
Iteration 3800
Iteration 3900
Iteration 4000
Iteration 4100
Iteration 4200
Iteration 4300
Iteration 4400
Iteration 4500
Iteration 4600
Iteration 4700
Iteration 4800
Iteration 4900
Iteration 5000
Kernel computed


In [ ]:
test = model.predict(np.array(train_data[5000:], dtype=object))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(train_labels_svm[5000], test))

## Save to csv

In [ ]:
# original labels are in {0, 1}
test_preds = test.copy()
test_preds[test_preds == -1] = 0
Yte = {'Prediction' : test_preds}
dataframe = pd.DataFrame(Yte) 
dataframe.index += 1 
dataframe.to_csv('test_pred.csv',index_label='Id')